# Weekly Assignment 4

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
import csv

# Classification with Naive Bayes

Loading data

In [2]:
class airbnb( object ):
    
    def __init__(self, latitude, longitude, neighbourhood, price, availability, room_type, minimum_nights):
        self.latitude=latitude
        self.longitude=longitude
        self.neighbourhood=neighbourhood
        self.price=price
        self.availability=availability
        self.room_type=room_type
        self.minimum_nights=minimum_nights
    
    def getAllFeatures(self): return {"latitude": self.latitude, "longitud": self.longitude, "neighbourhood":self.neighbourhood, "price": self.price, "availability": self.availability, "room_type": self.room_type, "minimum_nights": self.minimum_nights}
    
    def getFeaturesDict(self): return {"latitude": self.latitude, "longitud": self.longitude}
    
    def getNeighbour(self): return self.neighbourhood

In [3]:
myairbnbs = []
with open("listings.csv") as data:
    n = 1
    data = csv.reader(data)
    next(data)
    for line in data:
        apartment = airbnb(float(line[6]),float(line[7]),line[5],float(line[9]),int(line[-1]),line[8],int(line[10]))
        myairbnbs.append(apartment)

In [4]:
training_set = myairbnbs
test_set = myairbnbs[:100]

In [5]:
class NaiveBayesOfairnbns( object ):
 
    def _distributionProbFunction(self,neighbourhood, Property):
        # Return the result of evaluating a Normal distribution of:
        #   mean equal to the mean of the property for this type of fruit
        #   std  equal to the std  of the property for this type of fruit
        return lambda x: norm.pdf( x, self.cmean[neighbourhood][Property], self.cstdv[neighbourhood][Property] )
        
    def __init__(self, training_set):
        self.training_set = training_set
 
        self.neighbourhoods = {}
        for apartment in self.training_set:
            if apartment.getNeighbour() in self.neighbourhoods.keys():
                self.neighbourhoods[apartment.getNeighbour()] += 1
            else: self.neighbourhoods[apartment.getNeighbour()] = 1
 
        self.probl = {}
        self.N = len(self.training_set[0].getFeaturesDict())
        self.properties = list(self.training_set[0].getFeaturesDict().keys())
        self.cmean = {}
        self.cstdv = {}
        self.prob = {}
        
        for neighbourhood in self.neighbourhoods.keys():
            # The amount of fruits of each type that I have in my "training" dataset
            self.probl[neighbourhood] = self.neighbourhoods[neighbourhood]/len(self.training_set)
            
            # I will store the mean, std, and normal probabiblity function,
            # of each property for each type of fruit
            self.cmean[neighbourhood] = {} #[0 for i in range(self.N)]
            self.cstdv[neighbourhood] = {} #[0 for i in range(self.N)]
            self.prob[neighbourhood] = {} #[0 for i in range(self.N)]
            
            # Then, for each property
            for i, prop in enumerate(self.properties):
                # Make a numpy array taking the list of the read properties
                xi = np.array([float(e.getFeaturesDict()[prop]) for e in self.training_set if e.getNeighbour() == neighbourhood])
                
                # Fullfill the mean with the mean, the std with the std and the prob with the probability function
                self.cmean[neighbourhood][prop] = xi.mean()
                self.cstdv[neighbourhood][prop] = xi.std()
                self.prob[neighbourhood][prop] = self._distributionProbFunction(neighbourhood,prop)

    def predict(self, X):
        p = {}
        
        # In the moment of predicting I have the probabilities of each feature of belonging to each class.
        for neighbourhood in self.neighbourhoods.keys():
            vectorToMultiply = [self._distributionProbFunction(neighbourhood,prop)(X[i]) 
                                    for i, prop in enumerate(self.properties)
                                    if self._distributionProbFunction(neighbourhood,prop)(X[i]) != 0.0 ] 
            cp = np.prod( vectorToMultiply )
            
            p[neighbourhood] = self.probl[neighbourhood]*cp
            
        results = [(p[neighbourhood],neighbourhood) for neighbourhood in self.neighbourhoods.keys()]
        results.sort()
        return results[-1][1], results[-1][0]/sum([r[0] for r in results])

    def plotDistributions(self):
        for i, prop in enumerate(self.properties):
            nprops=len(range(self.N))
            rows = int(nprops/2)+int(nprops%2)
            plt.subplot(rows,2,i+1)
            for neighbourhood in list(self.neighbourhoods.keys()):
                mean = self.cmean[neighbourhood][prop]
                var = self.cstdv[neighbourhood][prop]
                x= np.linspace(mean-4*var,mean+4*var,200)
                xmin,xmax=plt.xlim()
                plt.xlim(min(xmin,mean-3*var),max(xmax,mean+3*var))
                plt.title(self.properties[i])
                
                self.prob[neighbourhood][prop] = self._distributionProbFunction(neighbourhood,prop)

                plt.plot(x,self.prob[neighbourhood][prop](x), label = neighbourhood)
                leg = plt.legend(loc="upper left")
                leg.get_frame().set_alpha(0.7)
        plt.show()
        
    def __str__(self):
        s = 'Gaussian Naive Bayes Model:\n'
        for l in self.neighbourhoods.keys():
            s+= ' P(%s) = %.3f\n'%(l, self.probl[l])
            for i,prop in enumerate(self.properties):
                s+= '  E(%s|%s) = %.3f\n'%(self.properties[i], l, self.cmean[l][prop])
                s+= '  VAR(%s|%s) = %.3f\n'%(self.properties[i], l, self.cstdv[l][prop]**2)
                s+= '\n'
        return s.strip()

In [6]:
myNBC = NaiveBayesOfairnbns(training_set)

In [7]:
myNBC.predict([400,40])

/home/arturo/.local/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1740: RuntimeWarning: divide by zero encountered in double_scalars
  x = np.asarray((x - loc)/scale, dtype=dtyp)


('Baró de Viver', nan)